# Section 1 : 

In [ ]:
with open('s.txt', encoding="utf8") as file:
     s = file.read()

In [ ]:
def remove_newlines_tabs(text):    
    # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
    Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ')
    return Formatted_text
s = remove_newlines_tabs(s)

In [ ]:
List = s.split("',")

In [ ]:
import pandas as pd 
df = pd.DataFrame(List, columns = ['sentence'])

In [ ]:
import spacy
from spacy.matcher import Matcher 

from spacy import displacy 
import visualise_spacy_tree
from IPython.display import Image, display

# load english language model
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

In [ ]:
# function for rule 1: noun(subject), verb, noun(object)
def rule1(text):
    
    doc = nlp(text)
    
    sent = []
    
    for token in doc:
        
        # if the token is a verb
        if (token.pos_=='VERB'):
            
            phrase =''
            
            # only extract noun or pronoun subjects
            for sub_tok in token.lefts:
                
                if (sub_tok.dep_ in ['nsubj','nsubjpass']) and (sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                    
                    # add subject to the phrase
                    phrase += sub_tok.text

                    # save the root of the verb in phrase
                    phrase += ' '+token.lemma_ 

                    # check for noun or pronoun direct objects
                    for sub_tok in token.rights:
                        
                        # save the object in the phrase
                        if (sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN']):
                                    
                            phrase += ' '+sub_tok.text
                            sent.append(phrase)
            
    return sent

In [ ]:
output_1 = rule1(plan_risk_management)

In [ ]:
output_1

['Management include processes',
 'risks affect project',
 'causes include requirement',
 'conditions include aspects',
 'risks assign reserve',
 'risk represent effect',
 'it include sources',
 'risk represent exposure',
 'organization accept risk',
 'organization tolerate risk',
 'attitude include appetite',
 'organization select response',
 'responses reflect organization',
 'planning enhance probability',
 'plan provide areas',
 'charter provide inputs',
 'register provide overview',
 'team allocate resources',
 'team allocate resources Judgment',
 'teams hold meetings',
 'Attendees include manager',
 'plan include following',
 'Roles define lead',
 'Roles define lead Defines',
 'categories provide means',
 'approaches use structure',
 'organization use framework',
 'formats describe content',
 'formats describe content reports',
 'formats describe content reports documents']

# Transformer : 

In [ ]:
with open('text1.txt', encoding="utf8") as file:
     plan_risk_management = file.read()

In [ ]:
def remove_newlines_tabs(text):    
    # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
    Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ')
    return Formatted_text
plan_risk_management = remove_newlines_tabs(plan_risk_management)

In [ ]:
text = plan_risk_management

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

STOPWORDS = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 

def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers.
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    


def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    df['clean_sentence'] = df['sentence'].apply(clean_text)
    df['tok_lem_sentence'] = df['clean_sentence'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True))
    return df
    
df = clean_sentences(df)

Cleaning sentences...


In [ ]:
df

,sentence,clean_sentence,tok_lem_sentence
0,['PROJECT RISK MANAGEMENT Project Risk Managem...,project risk management project risk managem...,"[project, risk, management, project, risk, man..."
1,'The objectives of project risk management a...,the objectives of project risk management a...,"[the, objective, of, project, risk, management..."
2,'Plan Risk Management is the process of defi...,plan risk management is the process of defi...,"[plan, risk, management, is, the, process, of,..."
3,'Identify Risks is the process of determinin...,identify risks is the process of determinin...,"[identify, risk, is, the, process, of, determi..."
4,'Perform Qualitative Risk Analysis is the pr...,perform qualitative risk analysis is the pr...,"[perform, qualitative, risk, analysis, is, the..."
...,...,...,...
98,'The specific combinations of probability an...,the specific combinations of probability an...,"[the, specific, combination, of, probability, ..."
99,'Revised stakeholders’ tolerances Stakeholde...,revised stakeholders tolerances stakeholde...,"[revised, stakeholder, tolerance, stakeholder,..."
100,'Reporting formats Reporting formats define ...,reporting formats reporting formats define ...,"[reporting, format, reporting, format, define,..."
101,'formats describes the content and format of...,formats describes the content and format of...,"[format, describes, the, content, and, format,..."


In [ ]:
def extract_best_indices(m, topk, mask=None):
    """
    Use sum of the cosine distance over all tokens.
    m (np.array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]  
    return best_index

In [ ]:
query_sentence = 'what is the project charter' 

pd.options.display.max_colwidth = 500

In [ ]:
! pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch


model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
corpus_embeddings = model.encode(df.sentence.values, convert_to_tensor=True)
query_embedding = model.encode(query_sentence, convert_to_tensor=True)

In [ ]:
import torch

# We use cosine-similarity and torch.topk to find the highest 3 scores
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=3)

print("\n\n======================\n\n")
print("Query:", query_sentence)
print("\nTop 5 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
    score = score.cpu().data.numpy() 
    idx = idx.cpu().data.numpy()
    display(df['sentence'].iloc[idx])





Query: what is the project charter

Top 5 most similar sentences in corpus:


"  '... Project Charter Described in Section ...."

"  'The project charter can provide various inputs such as high level risks, high level project descriptions, and high level requirements."

"  'The stakeholder register, which contains all details related to the project’s stakeholders, provides an overview of their roles."

In [ ]:
query_sentence_2 = 'what are the Analytical Techniques ? ' 

In [ ]:
corpus_embeddings = model.encode(df.sentence.values, convert_to_tensor=True)
query_embedding = model.encode(query_sentence_2, convert_to_tensor=True)

In [ ]:
import torch

# We use cosine-similarity and torch.topk to find the highest 3 scores
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=3)

print("\n\n======================\n\n")
print("Query:", query_sentence_2)
print("\nTop 5 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
    score = score.cpu().data.numpy() 
    idx = idx.cpu().data.numpy()
    display(df['sentence'].iloc[idx])





Query: what are the Analytical Techniques ? 

Top 5 most similar sentences in corpus:


"  '.. Plan Risk Management: Tools and Techniques ... Analytical Techniques Analytical techniques are used to understand and define the overall risk management context of the project."

"  'Perform Quantitative Risk Analysis is the process of numerically analyzing the effect of identified risks on overall project objectives."

"  'Other techniques, such as the use of strategic risk scoring sheets, are used to provide a high level assessment of the risk exposure of the project based on the overall project context."